<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [4]:
%%sql
SELECT *
FROM movies


 * sqlite:///movies.sqlite
Done.


imdb_key,title,production_year,running_time
tt0060196,"The Good, the Bad and the Ugly",1966,178
tt0482571,The Prestige,2006,130
tt0105236,Reservoir Dogs,1992,99
tt0108052,Schindlers List,1993,195


+ Show the performance dates for one of the movies.

In [5]:
%%sql
SELECT starting_date
FROM screenings
WHERE imdb_key = 'tt0108052'



 * sqlite:///movies.sqlite
Done.


starting_date
2022-02-10


+ Show all data concerning performances at a given theatere
  on a given date.

In [6]:
%%sql
SELECT starting_time AS 'Starting time',title AS 'Movie', running_time AS 'Movie length in minutes'
FROM screenings
join movies
using (imdb_key)
where theater_name = 'Filmstaden Martenstorg' AND starting_date = '2022-02-10'

 * sqlite:///movies.sqlite
Done.


Starting time,Movie,Movie length in minutes
18:00,"The Good, the Bad and the Ugly",178
21:00,The Prestige,130


+ List all customers

In [7]:
%%sql
SELECT username,customer_name
FROM customers

 * sqlite:///movies.sqlite
Done.


username,customer_name
Bippi,Bippi Sangstrump
Mippi,Mippi Tangstrump
pippi,Pippi Langstrump


+ List all tickets

In [10]:
%%sql
SELECT *
FROM tickets


 * sqlite:///movies.sqlite
Done.


ticket_id,screening_id,username
c432dc9628d2681a0a741a401e63435b,af6dd6efa272b6a18ad2efdc8361bed4,Bippi


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [11]:
%%sql
INSERT INTO tickets(screening_id,username)
VALUES ('af6dd6efa272b6a18ad2efdc8361bed4','Bippi')
RETURNING screening_id

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) cannot commit transaction - SQL statements in progress
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [14]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id
c432dc9628d2681a0a741a401e63435b


+ Try to insert two movie theaters with the same name (this
  should fail).

In [15]:
%%sql
INSERT INTO theaters(theater_name,capacity)
VALUES ('Filmstaden Martenstorg','300')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT INTO theaters(theater_name,capacity) VALUES ('Filmstaden Martenstorg','300')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [16]:
%%sql
INSERT INTO screenings(starting_date,starting_time,theater_name,imdb_key)
VALUES  ('2022-02-10','18:00','Filmstaden Karhuset','tt0060196')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screenings(starting_date,starting_time,theater_name,imdb_key) VALUES  ('2022-02-10','18:00','Filmstaden Karhuset','tt0060196')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [17]:
%%sql
INSERT INTO tickets(screening_id,username)
VALUES ('a','Bippi')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(screening_id,username) VALUES ('a','Bippi')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)